In [ ]:
import pandas as pd
path = 'data/FT_Camp_5/Train.csv'
data = pd.read_csv(path)
data.set_index('stockcode',inplace=True)
data.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold,cross_val_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
def up_sampling(X,y):
    '''
    上采样函数，运用SMOTE
    '''
    from imblearn.over_sampling import SMOTE
    sm = SMOTE(random_state=42)
    X,y=sm.fit_sample(X,y)
    return X,y
def view_ratio(X,y):
    '''
    查看正负类别比例
    '''
    n_sample = y.shape[0]
    n_pos_sample = y[y==0].shape[0]
    n_neg_sample = y[y==1].shape[0]
    print('样本个数：{}：正样本占{:.2%}：负样本占{:.2%}'.format(n_sample,n_pos_sample/n_sample,n_neg_sample/n_sample))
    print('特征维数:',X.shape[1])

In [ ]:
temp = data.isnull().any() #列中是否存在空值
print(type(temp))
print(temp)

In [ ]:
data.Invest_to_NA.fillna(0,inplace=True)

In [ ]:
X=data.iloc[:,0:34]
y=data.fake

In [ ]:
temp = X.isnull().any() #列中是否存在空值
print(type(temp))
print(temp)

In [ ]:
view_ratio(X,y)
X,y=up_sampling(X,y)
view_ratio(X,y)

In [ ]:
dtarin=xgb.DMatrix(X,label=y,missing=0)

In [ ]:
params_dict={
    'learning_rate':0.1,
    'n_estimators':816,
    'gamma':0,
    'max_depth':8,
    'min_child_weight':1,
    'colsample_bytree':0.9,
    'colsample_bylevel':1.0,
    'subsample':0.5,
    'reg_lambda':0.0,
    'reg_alpha':0.18,
    'seed':42
}
clf=xgb.XGBClassifier(**params_dict)
# cv_params={'n_estimators':np.linspace(750,850,10,dtype=int)}  
# cv_params={'max_depth':np.linspace(1,10,10,dtype=int)}
# cv_params={'min_child_weight':np.linspace(1,10,10,dtype=int)}
# cv_params={'gamma':np.linspace(0,0.1,11)}
# cv_params={'subsample':np.linspace(0,1,11)}
# cv_params={'colsample_bytree':np.linspace(0,1,11)[1:]}
# cv_params={'colsample_bylevel':np.linspace(0.9,1,11)[1:]}
# cv_params={'re_lambda':np.linspace(0,0.1,11)}
# cv_params={'reg_alpha':np.linspace(0.1,0.2,11)}
cv_params={'learning_rate':np.logspace(-2,0,11)}
gs = GridSearchCV(clf,cv_params,verbose=2,refit=True,cv=5,n_jobs=-1)
gs.fit(X,y)
print('best value:',gs.best_params_)
print('socre:',gs.best_score_)

In [ ]:
params_dict={
    'learning_rate':0.1,
    'n_estimators':816,
    'gamma':0,
    'max_depth':8,
    'min_child_weight':1,
    'colsample_bytree':0.9,
    'colsample_bylevel':1.0,
    'subsample':0.5,
    'reg_lambda':0.0,
    'reg_alpha':0.18,
    'seed':42
}

In [ ]:
def f1_cv(model,X,y):
    f1=cross_val_score(model,X,y,scoring='f1',cv=5)
    return f1
tuned_xgb=xgb.XGBClassifier(**params_dict)
score_t=f1_cv(tuned_xgb,X,y)
print('tuned:{:.4f}({:.4f})'.format(score_t.mean(),score_t.std()))

In [ ]:
test_path='data/FT_Camp_5/X_test.csv'
test_data=pd.read_csv(test_path)
test_data.set_index('stockcode',inplace=True)

In [ ]:
test_data.head()
temp = test_data.isnull().any() #列中是否存在空值
print(type(temp))
print(temp)

In [ ]:
test_data.Invest_to_NA.fillna(0,inplace=True)

In [ ]:
xgb_pred=xgb.XGBClassifier(**params_dict)
xgb_pred.fit(X,y)

In [ ]:
dtest = xgb.DMatrix(test_data, missing=0)

In [ ]:
test_data['fake']=xgb_pred.predict(test_data)

In [ ]:
save_=test_data.fake
save_.to_csv('sample_xgb.csv',sep=',',header=True,index=True,encoding='UTF-8')

In [ ]:
result=pd.read_csv('sample_xgb.csv')
result.describe()